<a href="https://colab.research.google.com/github/ev1025/DA_Study/blob/main/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%8C%80%EC%9A%A9%EB%9F%89csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dask 라이브러리

- 대용량 병렬처리방식, 메모리에 저장하지 않고 구조만 가져오는 방식
- .head()나 .tail()을 사용해야 데이터 확인가능

In [ ]:
pip install "dask[complete]"

In [ ]:
pip install openpyxl

In [ ]:
import dask.dataframe as dd
import json
import ast
import os
import sys
import csv

In [ ]:
# 현재 경로 확인
os.getcwd() 

In [ ]:
# 현재 경로 변경
os.chdir('D:\\train_v2.csv')

In [ ]:
# 대용량 csv파일 병렬처리
df = dd.read_csv('train_v2.csv', header='infer', engine='python',sep=',')

In [ ]:
#dask => pandas
df = ddf.compute()

In [ ]:
# pandas => dask
ddf = dd.from_pandas(df)

# CSV와 Excel

## CSV

In [ ]:
pd.read_csv('train_v2.csv',
            chunksize=10000, # 나눠서 불러올 때
            dtype={'fullVisitorId':'str',
                   ('date','fulVisitorid', 'visitId', 'visitNumber', 'visitStartTime'):'int'},
            converters={column: json.loads}, # json형식인 column이 str로 있는 경우
            nrows = n,      # 위에서 부터 가져올 행 개수
            skiprows=[1,2], # 위에서부터 1,2행 제외
            skipfooter = n, # 뒤에서부터 n개 제외
            usecols=[],     # 사용할 컬럼(숫자가능)
            name=[],        # 컬럼이름 설정(가져오는 개수와 일치해야함)
            sheet_name =[], # 시트명(숫자가능)
            header = None,   # 컬럼명 없을 때
            na_values = ['?', '??', 'N/A', 'NA', 'nan', 'NaN', '-nan', '-NaN', 'null') # 해당 값은 결측치로 인식

In [ ]:
# 내보내기
df.to_csv('경로\\파일.csv',
          index= False,
          lineterminator='\n', # 라인분리 제대로 하도록 해줌
          )

**대용량데이터 조금씩 꺼내서 열기**

In [ ]:
# chunksize만큼 꺼내기(바로 호출 불가)
address = pd.read_csv('train_v2.csv', chunksize=10000,
                       dtype={'fullVisitorId':'object',('date','fulVisitorid', 'visitId', 'visitNumber', 'visitStartTime'):'int'})
train_data = pd.DataFrame()

In [ ]:
# 1만개씩 30번 데이터 꺼내와서 빈 프레임에 넣기
i = 0
for row, chunk in enumerate(address):
    train_data = pd.concat([train_data, chunk])
    print(f'{row*10000}개')
    i+=1
    if i == 30:
        break

## EXCEL

In [ ]:
# 내보내기
ad = 'D:\\train_v2.csv\\'
df.to_excel(excel_writer= ad + '파일.xlsx', # 경로
            index=False)

In [ ]:
# 가져오기 (xls, xlsx, xlsm, xlsb, odf, ods and odt )
pd.read_execl('경로\\파일.xlsx',
              header = None )

## 에러

**필드값이 너무 큰 경우**
- field larger than field limit (131072)


In [ ]:
csv.field_size_limit(sys.maxsize)

- OverflowError: Python int too large to convert to C long

In [ ]:
# 필드의 값 제한 해제
maxparse = sys.maxsize
while True:
    try:
        csv.field_size_limit(maxparse)
        break
    except OverflowError:
        maxparse = int(maxparse/10)

### json이나 list가 str로 저장된 경우

- json은 반드시 [{"a":"b"}]같이 " "로 묶여있어야한다. ' '로 묶인 경우 안됨

In [ ]:
# '[{"a":"b"}]'  => [{"a":'b"}]
# str형식으로 된 값을 list로 바꾸기
ast.literal_eval('[{"a":"b"}]')

**컬럼값 전체를 변경해야 하는 경우**

In [ ]:
df['json_column'] = df['json_column'].apply(json.loads)

**' '로 묶인 경우**

In [ ]:
df['json_column'] = df['json_column'].str.replace("'", "\"")